# Lab 1-2: Train Hugging Face Transformers on Amazon SageMaker

### Multi-Class Classification with Naver Movie dataset and Hugging Face `Trainer` 
---

## Introduction
---

바로 이전 모듈에서 기존에 온프레미스에서 개발했던 환경과 동일한 환경으로 모델을 빌드하고 훈련했습니다. 하지만 아래와 같은 상황들에서도 기존 환경을 사용하는 것이 바람직할까요?

- 온프레미스의 GPU는 총 1장으로 훈련 시간이 너무 오래 소요됨
- 가용 서버 대수가 2대인데 10개의 딥러닝 모델을 동시에 훈련해야 함
- 필요한 상황에만 GPU를 활용 
- 기타 등등

Amazon SageMaker는 **데이터 과학자들 및 머신 러닝 엔지니어들을 위한 완전 관리형 머신 러닝 서비스**로 훈련 및 추론 수행 시 인프라 설정에 대한 추가 작업이 필요하지 있기에, 단일 GPU 기반의 딥러닝 훈련을 포함한 멀티 GPU 및 멀티 인스턴스 분산 훈련을 보다 쉽고 빠르게 수행할 수 있습니다. SageMaker는 다양한 유즈케이스들에 적합한 예제들을 지속적으로 업데이트하고 있으며, 한국어 세션 및 자료들도 제공되고 있습니다.

_**Note**_
- 이미 기본적인 Hugging Face 용법 및 자연어 처리에 익숙하신 분들은 앞 모듈을 생략하고 이 모듈부터 핸즈온을 시작하셔도 됩니다.
- 이 노트북은 SageMaker 기본 API를 참조하므로, SageMaker Studio, SageMaker 노트북 인스턴스 또는 AWS CLI가 설정된 로컬 시스템에서 실행해야 합니다. SageMaker Studio 또는 SageMaker 노트북 인스턴스를 사용하는 경우 PyTorch 기반 커널을 선택하세요.
- 훈련 job 수행 시 최소 `ml.p3.2xlarge` 이상의 훈련 인스턴스가 필요하며, `ml.p3.8xlarge`나 `ml.p3.16xlarge` 인스턴스를 권장합니다. 만약 인스턴스 사용에 제한이 걸려 있다면 [Request a service quota increase for SageMaker resources](https://docs.aws.amazon.com/sagemaker/latest/dg/regions-quotas.html#service-limit-increase-request-procedure)를 참조하여 인스턴스 제한을 해제해 주세요.


# Development Environment and Permissions 

## Installation

이 예제 노트북은 **SageMaker Python SDK v2.70.0** 이상과 **transformers v4.11.0** 이상이 필요합니다.

이 노트북을 처음 실행하신다면, 아래 코드 셀에서 `install_needed = True`로 변경 후, 코드 셀을 실행하세요. 패키지 인스톨 이후 노트북 커널이 재시작되며, `install_needed = False`로 변경 후, 코드 셀을 다시 실행합니다. 이 작업은 한 번만 수행하면 됩니다.

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import IPython

#install_needed = True
install_needed = False

if install_needed:
    print("===> Installing deps and restarting kernel. Please change 'install_needed = False' and run this code cell again.")
    !{sys.executable} -m pip install -U "sagemaker>=2.70.0" "transformers>=4.11.0" "s3fs"
    IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import boto3
import sagemaker
import sagemaker.huggingface

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
import os
import sys
import logging

logging.basicConfig(
    level=logging.INFO, 
    format='[{%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

<br>

## 1. Preparation
---

In [ ]:
from datasets import load_dataset
from transformers import ElectraModel, ElectraTokenizer

# Define the model repo
tokenizer_id = 'monologg/koelectra-small-v3-discriminator'
model_id = 'monologg/koelectra-small-v3-discriminator'

# dataset used
dataset_name = 'nsmc'

# s3 key prefix for the data
s3_prefix = 'samples/datasets/nsmc'

### Dataset

본 핸즈온에서 사용할 말뭉치 데이터셋은 네이버 영화 리뷰 데이터(https://github.com/e9t/nsmc/) 공개 데이터셋으로 15만 건의 훈련 데이터와 5만 건의 테스트 데이터로 구성되어 있습니다. 이 데이터셋은 한국어 자연어 처리 모델 벤치마킹에 자주 사용됩니다.

![emotion-widget.png](../imgs/nsmc-classification.png)

In [ ]:
# load dataset
train_dataset, test_dataset = load_dataset(dataset_name, split=['train', 'test'])

# num_samples_for_debug = 2000
# train_dataset = train_dataset.shuffle(seed=42).select(range(num_samples_for_debug))
# test_dataset = test_dataset.shuffle(seed=42).select(range(num_samples_for_debug))

logger.info(f" loaded train_dataset length is: {len(train_dataset)}")
logger.info(f" loaded test_dataset length is: {len(test_dataset)}")
logger.info(train_dataset[0])

### Tokenization

자연어 처리 모델을 훈련하려면, 토큰화(Tokenization)를 통해 말뭉치(corpus; 자연어 처리를 위한 대량의 텍스트 데이터)를 토큰 시퀀스로 나누는 과정이 필요합니다. BERT 이전의 자연어 처리 모델은 주로 도메인 전문가들이 직접 토큰화해놓은 토크아니저(Mecab, Kkma 등)들을 사용했지만, BERT를 훈련하기 위한 토크나이저는 도메인 지식 필요 없이 말뭉치에서 자주 등장하는 서브워드(subword)를 토큰화합니다. GPT 기반 모델은 BPE(Byte-pair Encoding)라는 통계적 기법을 사용하며, BERT 및 ELECTRA 기반 모델은 BPE와 유사한 Wordpiece를 토크나이저로 사용합니다.

In [ ]:
# download tokenizer
tokenizer = ElectraTokenizer.from_pretrained(tokenizer_id)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['document'], padding='max_length', truncation=True)

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# set format for pytorch
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

### Uploading data to Amazon S3 Bucket

SageMaker 훈련을 위해 `FileSystem`를 사용하여 말뭉치 데이터셋을 S3로 업도르합니다. 자세한 내용은 https://huggingface.co/docs/datasets/filesystems.html 를 참조해 주세요.

In [ ]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()  

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path, fs=s3)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path, fs=s3)

<br>

## 2. Training with Native Hugging Face (PyTorch Framework)

---

### Overview and Training Script

SageMaker에 대한 대표적인 오해가 여전히 많은 분들이 SageMaker 훈련을 위해 소스 코드를 전면적으로 수정해야 한다고 생각합니다. 하지만, 실제로는 별도의 소스 코드 수정 없이 기존 여러분이 사용했던 파이썬 스크립트에 SageMaker 훈련에 필요한 SageMaker 전용 환경 변수들만 추가하면 됩니다. 

SageMaker 훈련은 훈련 작업을 호출할 때, 1) 훈련 EC2 인스턴스 프로비저닝 - 2) 컨테이너 구동을 위한 도커 이미지 및 훈련 데이터 다운로드 - 3) 컨테이너 구동 - 4) 컨테이너 환경에서 훈련 수행 - 5) 컨테이너 환경에서 S3의 특정 버킷에 저장 - 6) 훈련 인스턴스 종료로 구성됩니다. 따라서, 훈련 수행 로직은 아래 예시와 같이 기존 개발 환경과 동일합니다.

```python
/opt/conda/bin/python train.py --epochs 5 --train_batch_size 32 ...
```

이 과정에서 컨테이너 환경에 필요한 환경 변수(예: 모델 경로, 훈련 데이터 경로) 들은 사전에 지정되어 있으며, 이 환경 변수들이 설정되어 있어야 훈련에 필요한 파일들의 경로를 인식할 수 있습니다. 대표적인 환경 변수들에 대한 자세한 내용은 https://github.com/aws/sagemaker-containers#important-environment-variables 을 참조하세요.

이 과정을 다이어그램으로 나타내면 아래와 같습니다.

![SageMaker Hugging Face Diagram](../imgs/sm-hf-diagram.png)

또한, 핸즈온을 위해 미리 작성된 훈련 스크립트를 확인해 보세요. 종래 코드와 동일하다는 것을 알 수 있습니다.

In [ ]:
# !pygmentize ./scripts/train.py

In [ ]:
from sagemaker.huggingface import HuggingFace
import time
instance_type = 'ml.p3.8xlarge'
num_gpus = 4
instance_count = 1
batch_size_native = 32
learning_rate_native = float('5e-5')
batch_size = batch_size_native
learning_rate = learning_rate_native / batch_size_native * batch_size * num_gpus * instance_count
logger.info(learning_rate)

# hyperparameters, which are passed into the training job
hyperparameters = {
    'n_gpus': num_gpus,                 # number of GPUs per instance
    'epochs': 3,                        # number of training epochs
    'seed': 42,                         # seed
    'train_batch_size': batch_size,     # batch size for training
    'eval_batch_size': batch_size*2,    # batch size for evaluation
    'warmup_steps': 0,                  # warmup steps
    'learning_rate': learning_rate,     # learning rate used during training
    'tokenizer_id': model_id,           # pre-trained tokenizer
    'model_id': model_id                # pre-trained model
}

In [ ]:
# define Training Job Name 
job_name = f'huggingface-native-pytorch-190-hol-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
chkpt_s3_path = f's3://{sess.default_bucket()}/{s3_prefix}/native/checkpoints'

# create the Estimator
sm_estimator = HuggingFace(
    entry_point           = 'train.py',         # fine-tuning script used in training jon
    source_dir            = './scripts',        # directory where fine-tuning script is stored
    instance_type         = instance_type,      # instances type used for the training job
    instance_count        = instance_count,     # the number of instances used for training
    base_job_name         = job_name,           # the name of the training job
    role                  = role,               # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version  = '4.11.0',           # the transformers version used in the training job
    pytorch_version       = '1.9.0',            # the pytorch_version version used in the training job
    py_version            = 'py38',             # the python version used in the training job
    hyperparameters       = hyperparameters,    # the hyperparameter used for running the training job
    disable_profiler      = True,
    debugger_hook_config  = False, 
    checkpoint_s3_uri     = chkpt_s3_path,
    checkpoint_local_path ='/opt/ml/checkpoints',  
)

`fit()` 메소드를 호출하여 훈련 job을 시작합니다. `fit()` 메소드의 인자값 중 `wait=True`로 설정할 경우에는 동기(synchronous) 방식으로 동직하게 되며, `wait=False`일 경우에는 비동기(aynchronous) 방식으로 동작하여 여러 개의 훈련 job을 동시에 실행할 수 있습니다.

In [ ]:
# define a data input dictonary with our uploaded s3 uris
data = {
    'train': training_input_path,
    'test': test_input_path
}

# starting the train job with our uploaded datasets as input
sm_estimator.fit(data, wait=False)
train_job_name = sm_estimator.latest_training_job.job_name

<br>

## 3. Training with Optimized Hugging Face (PyTorch Framework)

---

SageMaker Training Compiler는 별도의 추가 비용 없이 모델의 메모리 사용률을 감소시킴으로써(즉, 가용 미니배치 크기가 증가함으로써), GPU 인스턴스의 훈련 시간을 단축할 수 있는 신규 서비스입니다. SageMaker Training Compiler를 지원하는 AWS Deep Learning Containers(DLC)를 사용하면 코드 변경을 최소화하면서 `TrainingCompilerConfig()` 설정으로 단일 GPU부터 분산 훈련까지 편리하게 훈련 작업의 속도를 가속화할 수 있습니다.

예를 들어, 아래 코드 셀에서 `batch_size`가 32에서 48로 변경되었습니다. Training Compiler를 적용하지 않는다면, GPU 메모리에 모델과 미니배치가 들어갈 수 없으므로 OOM(Out of Memory) 오류가 발생합니다.

자세한 내용은 Amazon SageMaker 개발자 안내서의 SageMaker Training Compiler를 참조하세요.

**Note: SageMaker Training Compiler 적용 시에는 추가 오버헤드를 피하기 위해 가급적 SageMaker Debugger 기능을 끄는 것을 권장합니다.**

In [ ]:
from sagemaker.huggingface import HuggingFace, TrainingCompilerConfig
import time
instance_type = 'ml.p3.8xlarge'
num_gpus = 4
instance_count = 1
batch_size_native = 32
learning_rate_native = float('5e-5')
batch_size = 48
learning_rate = learning_rate_native / batch_size_native * batch_size * num_gpus * instance_count

# hyperparameters, which are passed into the training job
hyperparameters = {
    'training_script': 'train.py',      # training scripts
    'n_gpus': num_gpus,                 # number of GPUs per instance
    'epochs': 3,                        # number of training epochs
    'seed': 42,                         # seed
    'train_batch_size': batch_size,     # batch size for training
    'eval_batch_size': batch_size*2,    # batch size for evaluation
    'warmup_steps': 0,                  # warmup steps
    'learning_rate': learning_rate,     # learning rate used during training
    'tokenizer_id': model_id,           # pre-trained tokenizer
    'model_id': model_id                # pre-trained model
}

In [ ]:
# define Training Job Name 
job_name = f'huggingface-compiled-pytorch-190-hol-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
chkpt_s3_path = f's3://{sess.default_bucket()}/{s3_prefix}/compiler/checkpoints'

# create the Estimator
sm_compiler_estimator = HuggingFace(
    entry_point          = 'distributed_training_launcher.py',  
    source_dir           = './scripts',         # directory where fine-tuning script is stored
    instance_type        = instance_type,       # instances type used for the training job
    instance_count       = instance_count,      # the number of instances used for training
    base_job_name        = job_name,            # the name of the training job
    role                 = role,                # IAM role used in training job to access AWS ressources, e.g. S3
    transformers_version = '4.11.0',            # the transformers version used in the training job
    pytorch_version      = '1.9.0',             # the pytorch_version version used in the training job
    py_version           = 'py38',              # the python version used in the training job
    hyperparameters      = hyperparameters,     # the hyperparameter used for running the training job,
    compiler_config      = TrainingCompilerConfig(),
    disable_profiler     = True,
    debugger_hook_config = False,
    checkpoint_s3_uri    = chkpt_s3_path,
    checkpoint_local_path= '/opt/ml/checkpoints'    
)

`fit()` 메소드를 호출하여 훈련 job을 시작합니다. `fit()` 메소드의 인자값 중 `wait=True`로 설정할 경우에는 동기(synchronous) 방식으로 동직하게 되며, `wait=False`일 경우에는 비동기(aynchronous) 방식으로 동작하여 여러 개의 훈련 job을 동시에 실행할 수 있습니다.

In [ ]:
# starting the train job with our uploaded datasets as input
sm_compiler_estimator.fit(data, wait=False)
compiler_train_job_name = sm_compiler_estimator.latest_training_job.job_name

In [ ]:
train_job_name, compiler_train_job_name

### View Training Job
SageMaker 콘솔 창에서 훈련 내역을 직접 확인할 수도 있지만, 아래 코드 셀에서 생성되는 링크를 클릭하면 더 편리하게 훈련 내역을 확인할 수 있습니다.

In [ ]:
from IPython.core.display import display, HTML

def make_console_link(region, train_job_name, train_task='[Training]'):
    train_job_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{train_job_name}">Training Job</a></b>'   
    cloudwatch_link = f'<b> {train_task} Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={region}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={train_job_name};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a></b>'
    return train_job_link, cloudwatch_link  
        
train_job_link, cloudwatch_link = make_console_link(region, train_job_name, '[Hugging Face Training - Native]')
compiler_train_job_link, compiler_cloudwatch_link = make_console_link(region, compiler_train_job_name, '[Hugging Face Training - Compiler]')

display(HTML(train_job_link))
display(HTML(cloudwatch_link))
display(HTML(compiler_train_job_link))
display(HTML(compiler_cloudwatch_link))

### Wait for the training jobs to complete

훈련이 완료될 때까지 기다립니다. `estimator.fit(...)`에서 `wait=False`로 설정한 경우 아래 코드 셀의 주석을 해제 후 실행하여 동기 방식으로 변경할 수도 있습니다. 훈련 완료까지는 수십 분의 시간이 소요됩니다.

In [ ]:
# sess.logs_for_job(job_name=train_job_name, wait=True)

In [ ]:
sess.logs_for_job(job_name=compiler_train_job_name, wait=True)

In [ ]:
# waiter = sm_estimator.sagemaker_session.sagemaker_client.get_waiter(
#     "training_job_completed_or_stopped"
# )
# waiter.wait(TrainingJobName=train_job_name)
# waiter2 = sm_compiler_estimator.sagemaker_session.sagemaker_client.get_waiter(
#     "training_job_completed_or_stopped"
# )
# waiter2.wait(TrainingJobName=compiler_train_job_name)

<br>

## 4. Analysis
---

### Create helper functions for analysis

In [ ]:
from ast import literal_eval
from collections import defaultdict
from matplotlib import pyplot as plt

# Intermediary function for processing each line of stdout captured
# Remove leading and trailing whitespace and append data in curly braces
# to final list
def _summarize(captured):
    final = []
    for line in captured.stdout.split("\n"):
        cleaned = line.strip()
        if "{" in cleaned and "}" in cleaned:
            final.append(cleaned[cleaned.index("{") : cleaned.index("}") + 1])
    return final


# Check input with literal_eval
# https://docs.python.org/3/library/ast.html
def make_sense(string):
    try:
        return literal_eval(string)
    except:
        pass


# Parse the stdout and organize by train, evaluation, and summary data
def summarize(summary):
    final = {"train": [], "eval": [], "summary": {}}
    for line in summary:
        interpretation = make_sense(line)
        if interpretation:
            if "loss" in interpretation:
                final["train"].append(interpretation)
            elif "eval_loss" in interpretation:
                final["eval"].append(interpretation)
            elif "train_runtime" in interpretation:
                final["summary"].update(interpretation)
    return final

In [ ]:
%%capture native

# access the logs of the native training job
sm_estimator.sagemaker_session.logs_for_job(train_job_name)

In [ ]:
%%capture optimized

# access the logs of the native training job
sm_compiler_estimator.sagemaker_session.logs_for_job(compiler_train_job_name)

In [ ]:
native_summary = summarize(_summarize(native))
native_throughput = native_summary["summary"]["train_samples_per_second"]

optimized_summary = summarize(_summarize(optimized))
optimized_throughput = optimized_summary["summary"]["train_samples_per_second"]

avg_speedup = f"{round((optimized_throughput/native_throughput-1)*100)}%"

### Plot Optimized vs Native Training Throughput

Visualize average throughputs as reported by HuggingFace and see potential savings.

In [ ]:
# Average throughput for the native PyTorch training as reported by Trainer
n = summarize(_summarize(native))
native_throughput = n["summary"]["train_samples_per_second"]

# Average throughput for the optimized PyTorch training as reported by Trainer
o = summarize(_summarize(optimized))
optimized_throughput = o["summary"]["train_samples_per_second"]

# Calculate percentage speedup of optimized PyTorch over native PyTorch
avg_speedup = f"{round((optimized_throughput/native_throughput-1)*100)}%"

In [ ]:
%matplotlib inline

plt.title("Training Throughput \n (Higher is better)")
plt.ylabel("Samples/sec")

plt.bar(x=[1], height=native_throughput, label="Baseline PT", width=0.35)
plt.bar(x=[1.5], height=optimized_throughput, label="Compiler-enhanced PT", width=0.35)

plt.xlabel("  ====> {} Compiler savings <====".format(avg_speedup))
plt.xticks(ticks=[1, 1.5], labels=["Baseline PT", "Compiler-enhanced PT"])

In [ ]:
native_summary['train']

In [ ]:
vanilla_loss = [i["loss"] for i in n["train"]]
vanilla_epochs = [i["epoch"] for i in n["train"]]
optimized_loss = [i["loss"] for i in o["train"]]
optimized_epochs = [i["epoch"] for i in o["train"]]

plt.title("Plot of Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Training Loss")
plt.plot(vanilla_epochs, vanilla_loss, label="Baseline PT")
plt.plot(optimized_epochs, optimized_loss, label="Compiler-enhanced PT")
plt.legend()

### Training Stats

SageMaker Training Compiler가 있는 경우와 없는 경우 다양한 훈련 메트릭을 비교해 보겠습니다. SageMaker Training Compiler는 훈련 처리량을 증가시켜 총 훈련 시간을 줄여줍니다.

In [ ]:
import pandas as pd

pd.DataFrame([n["summary"], o["summary"]], index=["Native", "Optimized"])

In [ ]:
# calculate percentage speedup from SageMaker Training Compiler in terms of total training time reported by HF
speedup = (
    (n["summary"]["train_runtime"] - o["summary"]["train_runtime"])
    * 100
    / n["summary"]["train_runtime"]
)
print(
    f"SageMaker Training Compiler integrated PyTorch is about {int(speedup)}% faster in terms of total training time as reported by HF."
)

In [ ]:
avg_speedup = f"{round((optimized_throughput/native_throughput-1)*100)}%"
print(avg_speedup)

In [ ]:
def BillableTimeInSeconds(name):
    describe_training_job = (
        sm_compiler_estimator.sagemaker_session.sagemaker_client.describe_training_job
    )
    details = describe_training_job(TrainingJobName=name)
    return details["BillableTimeInSeconds"]

Billable = {}
Billable["Native"] = BillableTimeInSeconds(train_job_name)
Billable["Optimized"] = BillableTimeInSeconds(compiler_train_job_name)
display(pd.DataFrame(Billable, index=["BillableSecs"]))

speedup = (Billable["Native"] - Billable["Optimized"]) * 100 / Billable["Native"]
print(f"SageMaker Training Compiler integrated PyTorch was {int(speedup)}% faster in summary.")

<br>

## 5. Clean up
---

훈련 작업이 아직 실행 중인 경우, 과금 방지를 위해 시작된 모든 훈련 작업을 중단합니다.

In [ ]:
import boto3
sm = boto3.client("sagemaker")

def stop_training_job(name):
    status = sm.describe_training_job(TrainingJobName=name)["TrainingJobStatus"]
    if status == "InProgress":
        sm.stop_training_job(TrainingJobName=name)


stop_training_job(train_job_name)
stop_training_job(compiler_train_job_name)